In [7]:
#basic imports
import pandas as pd
import numpy as np
import math
#ml stuff
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Model
from keras.models import Sequential
#metrics + sklearn
from scipy import spatial
from hdbscan import HDBSCAN
from sklearn import metrics
from sklearn import decomposition
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import numpy as np 
#data visualization
import plotly.graph_objs as go
import plotly
import hiplot as hip

C:\Users\madke\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
#aware merge will strip down to 120
aware = pd.read_csv('/users/madke/downloads/aware_personality_8-4.csv', index_col = 0)
awaregb = aware.groupby('participantID')


aware['localDate'] =pd.to_datetime(aware.localDate)
aware = aware.loc[:, aware.isna().mean() < .40]

sequences = []
for i in list(set(aware['participantID'])):
    unit = awaregb.get_group(i)
    unit = unit.sort_values(by='localDate')
    unit = unit.reset_index(drop = True)
    sequences.append(unit.reset_index(drop = False))


imp_seq = []
for i in sequences:
    needs_fixed = i.iloc[:,2:(len(i.columns)-10)]
    fixed = needs_fixed.fillna(needs_fixed.mean())
    fixed = fixed.fillna(0)
    new = pd.concat([i.iloc[:,0:2], fixed, i.iloc[:, (len(i.columns)-10):(len(i.columns))]], axis = 1)
    imp_seq.append(new)
  

In [9]:
dep_vars = []
indep_vars = []
for i in imp_seq:
    dep = i.iloc[:, (len(i.columns)-10):(len(i.columns))]
    dep = dep.iloc[:1, :]
    dep_vars.append(np.array(dep))
    indep = i.iloc[:, 0:len(i.columns)-10].drop(['participantID', 'localDate'], axis = 1)
    indep_vars.append(indep)
#here is padding the values for nns (same size 90th percentile)
pad_imp = []
for i in indep_vars:
    if len(i['index']) >= 21:
        pad_imp.append(np.array(i.iloc[0:21,:]))
    else:
        count = 21 - len(i['index'])
        for z in range(0,count):
            i.loc[len(i)] = 0
        pad_imp.append(np.array(i))
    

In [31]:
# split a  sequence into samples
#https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
def split_sequence(sequence, n_steps = 3):
    X, y = list(), list()
    for i in range(sequence.shape[0]):
# find the end of this pattern
        end_ix = i + n_steps
    # check if we are beyond the sequence
        if end_ix > sequence.shape[0] -1:
            break
            # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix, :], sequence[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
 


In [120]:
X_ls = []
y_ls = []
for i in pad_imp:
    X, y = split_sequence(i)
    X_ls.append(X)
    y_ls.append(y)

In [121]:
X_ls[1].shape

(18, 3, 385)

In [124]:
dep_vars_ls = y_ls
indep_vars_ls =X_ls
indep_vars_ls = np.where(np.isnan(indep_vars_ls),0,indep_vars_ls)


y = np.vstack(dep_vars_ls)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
y = scaler.fit_transform(y)

#y = y.reshape(120,18,385)
y = np.where(np.isnan(y),0,y)

# normalize the dataset
X = np.array(indep_vars_ls)
scaled = np.vstack(X)

print(y.shape)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaled.reshape(2160,3,385)
X = scaled
X = np.where(np.isnan(X),0,X)
#[samples, timesteps, features]

(2160, 385)


In [98]:
y.shape

(2160, 385)

In [125]:

n_steps, n_features = X.shape[1], X.shape[2]
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(385, input_shape=(n_steps, n_features)))
model.compile(optimizer='adam', loss='mse')

In [126]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, 50)                87200     
_________________________________________________________________
dense_13 (Dense)             (None, 385)               19635     
Total params: 106,835
Trainable params: 106,835
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, y, epochs=200, verbose=0)

In [77]:
np.random.seed(300)
tf.compat.v1.random.set_random_seed(2005)

learn = 0.01
opt = tf.optimizers.Adam(lr= learn, amsgrad = True)
#opt = tf.optimizers.Adadelta(lr= learn)

loss_fctn = 'mean_squared_error'
activ_h = 'tanh'  #selu, tanh
KI_h = 'glorot_uniform'

model = Sequential()
model.add(Dense(100, input_shape=(X_train.shape[1], X_train.shape[2]), activation=activ_h, kernel_initializer=KI_h))
#model.add(Dense(50, input_shape=(X_train.shape[1], X_train.shape[2]), activation=activ_h, kernel_initializer=KI_h))
#model.add(Dense(25, input_shape=(X_train.shape[1], X_train.shape[2]), activation=activ_h, kernel_initializer=KI_h))
model.add(LSTM(32, activation=activ_h, kernel_initializer=KI_h, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), recurrent_dropout = 0.4,dropout=0.4))
model.add(LSTM(32, activation=activ_h, kernel_initializer=KI_h, return_sequences=True, recurrent_dropout = 0.2,   dropout=0.2))
model.add(LSTM(32, activation=activ_h, kernel_initializer=KI_h, return_sequences=True, recurrent_dropout = 0.2,   dropout=0.2))
model.add(LSTM(32))
model.add(Dense(16, input_dim=32, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss=loss_fctn, optimizer=opt,  metrics=['acc', 'mae', 'msle', 'mse'])
 


NameError: name 'X_train' is not defined